In [1]:
!pip install roboflow ultralytics

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datasets 

In [2]:
!pip install --force-reinstall numpy matplotlib --upgrade

  Using cached numpy-2.3.5-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.7-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pillow-12.0.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cache

In [3]:
from roboflow import Roboflow
from ultralytics import YOLO

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("ROBOFLOW_API_KEY")

In [5]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("new1-2zcyf").project("fall_detaction")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fall_detaction-1 in yolov8:: 100%|██████████| 18520/18520 [00:02<00:00, 6547.78it/s] 


In [6]:
%%writefile /kaggle/working/fall_detaction-1/data.yaml
train: /kaggle/working/fall_detaction-1/train/images
val: /kaggle/working/fall_detaction-1/valid/images
test: /kaggle/working/fall_detaction-1/test/images

nc: 3
names: ['stand', 'falling', 'fallen']


Overwriting /kaggle/working/fall_detaction-1/data.yaml


In [8]:
# Load the base model
model = YOLO("yolov8n.pt")

# Train with wandb logging enabled
results = model.train(
    data="/kaggle/working/fall_detaction-1/data.yaml",
    epochs=200,
    imgsz=640,
    device=0,
    batch=8,
)

Ultralytics 8.3.230 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/fall_detaction-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

In [10]:
success = model.export(format='onnx')
model_v8_tf = model.export(format='tflite')

Ultralytics 8.3.230 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)

ONNX: starting export with onnx 1.18.0 opset 19...
ONNX: slimming with onnxslim 0.1.75...
ONNX: export success ✅ 1.0s, saved as '/kaggle/working/runs/detect/train2/weights/best.onnx' (11.7 MB)

Export complete (1.3s)
Results saved to /kaggle/working/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640 data=/kaggle/working/fall_detaction-1/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.230 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
Model 

I0000 00:00:1763912476.443927     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13432 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763912476.444818     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
E0000 00:00:1763912480.395259     128 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1763912480.848631     128 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs t

Saved artifact at '/kaggle/working/runs/detect/train2/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 7, 8400), dtype=tf.float32, name=None)
Captures:
  132173235392144: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  132173235391760: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  132173235392528: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  132173235394256: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  132173235396944: TensorSpec(shape=(3, 3, 16, 32), dtype=tf.float32, name=None)
  132173235396560: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  132173235397520: TensorSpec(shape=(1, 1, 32, 32), dtype=tf.float32, name=None)
  132173235397904: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  132173235398480: TensorSpec(shape=(4,), dtype=tf.int64, name=None

I0000 00:00:1763912483.691732     128 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
I0000 00:00:1763912483.691887     128 single_machine.cc:361] Starting new session
I0000 00:00:1763912483.693978     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13432 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763912483.694187     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
W0000 00:00:1763912484.373034     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763912484.373083     128 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1763912485.106430     128 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
I0000 00:00:1763912485.106

TensorFlow SavedModel: export success ✅ 14.9s, saved as '/kaggle/working/runs/detect/train2/weights/best_saved_model' (29.5 MB)

TensorFlow Lite: starting export with tensorflow 2.18.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/kaggle/working/runs/detect/train2/weights/best_saved_model/best_float32.tflite' (11.7 MB)

Export complete (15.3s)
Results saved to /kaggle/working/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train2/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train2/weights/best_saved_model/best_float32.tflite imgsz=640 data=/kaggle/working/fall_detaction-1/data.yaml  
Visualize:       https://netron.app


In [11]:
success = model.export(format='onnx')

Ultralytics 8.3.230 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)

ONNX: starting export with onnx 1.18.0 opset 19...
ONNX: slimming with onnxslim 0.1.75...
ONNX: export success ✅ 0.9s, saved as '/kaggle/working/runs/detect/train2/weights/best.onnx' (11.7 MB)

Export complete (1.3s)
Results saved to /kaggle/working/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640 data=/kaggle/working/fall_detaction-1/data.yaml  
Visualize:       https://netron.app


In [12]:
success = model.export(format='tflit', int8=True)

WARNING ⚠️ Invalid export format='tflit', updating to format='tflite'
Ultralytics 8.3.230 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)

TensorFlow SavedModel: starting export with tensorflow 2.18.0...
TensorFlow SavedModel: collecting INT8 calibration images from 'data=coco8.yaml'

WARNING ⚠️ Dataset 'coco8.yaml' images not found, missing path '/kaggle/working/datasets/coco8/images/val'
Unzipping /kaggle/working/datasets/coco8.zip to /kaggle/working/datasets/coco8...: 100% ━━━━━━━━━━━━ 25/25 4.6Kfiles/s 0.0s
Dataset download success ✅ (0.3s), saved to /kaggle/working/datasets

Fast image access ✅ (ping: 0.0±0.0 ms, read: 1216.0±37

E0000 00:00:1763912854.178198     128 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1763912854.645120     128 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1763912855.025296     128 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and 

Saved artifact at '/kaggle/working/runs/detect/train2/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 7, 8400), dtype=tf.float32, name=None)
Captures:
  132172373913872: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  132172373911184: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  132172373911568: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  132172373910416: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  132172373911376: TensorSpec(shape=(3, 3, 16, 32), dtype=tf.float32, name=None)
  132172373910800: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  132172373913104: TensorSpec(shape=(1, 1, 32, 32), dtype=tf.float32, name=None)
  132172373914448: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  132172373914832: TensorSpec(shape=(4,), dtype=tf.int64, name=None

I0000 00:00:1763912857.234794     128 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
I0000 00:00:1763912857.234945     128 single_machine.cc:361] Starting new session
I0000 00:00:1763912857.237082     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13432 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763912857.237289     128 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
W0000 00:00:1763912857.904765     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763912857.904805     128 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1763912858.508201     128 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
I0000 00:00:1763912858.508

W0000 00:00:1763912863.076423     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763912863.076466     128 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1763912863.142417     128 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
W0000 00:00:1763912869.077203     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763912869.077239     128 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
W0000 00:00:1763912874.739108     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763912874.739143     128 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
W0000 00:00:1763912883.728204     128 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_for

TensorFlow SavedModel: export success ✅ 42.1s, saved as '/kaggle/working/runs/detect/train2/weights/best_saved_model' (38.9 MB)

TensorFlow Lite: starting export with tensorflow 2.18.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/kaggle/working/runs/detect/train2/weights/best_saved_model/best_int8.tflite' (3.2 MB)

Export complete (42.4s)
Results saved to /kaggle/working/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train2/weights/best_saved_model/best_int8.tflite imgsz=640 int8 
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train2/weights/best_saved_model/best_int8.tflite imgsz=640 data=/kaggle/working/fall_detaction-1/data.yaml int8 
Visualize:       https://netron.app
